## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-04-02-02-49 +0000,nyt,"Medieval Tower in Rome Partially Collapses, Tr...",https://www.nytimes.com/2025/11/03/world/europ...
1,2025-11-04-02-00-00 +0000,wsj,What Happened When Small-Town America Became D...,https://www.wsj.com/us-news/what-happened-when...
2,2025-11-04-02-00-00 +0000,wsj,Wall Street Intensifies Scrutiny of Fraud Afte...,https://www.wsj.com/finance/investing/wall-str...
3,2025-11-04-02-00-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...
4,2025-11-04-01-56-30 +0000,nypost,Andrew Cuomo backed by NYC Satmar Hasidic comm...,https://nypost.com/2025/11/03/us-news/andrew-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,45
28,will,17
181,new,15
63,china,13
238,deal,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
62,2025-11-03-22-40-36 +0000,nypost,Trump tells New York ‘you must vote’ for Andre...,https://nypost.com/2025/11/03/us-news/trump-te...,111
277,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,107
3,2025-11-04-02-00-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...,97
136,2025-11-03-19-19-01 +0000,nyt,Will Trump’s Tariff Deal Tilt the Playing Fiel...,https://www.nytimes.com/2025/11/02/business/ec...,93
146,2025-11-03-18-56-35 +0000,bbc,Trump administration will tap emergency fund t...,https://www.bbc.com/news/articles/cglg632n719o...,92


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
62,111,2025-11-03-22-40-36 +0000,nypost,Trump tells New York ‘you must vote’ for Andre...,https://nypost.com/2025/11/03/us-news/trump-te...
277,62,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...
203,49,2025-11-03-15-44-45 +0000,nypost,Kimberly-Clark buys Tylenol maker Kenvue in a ...,https://nypost.com/2025/11/03/business/tylenol...
210,44,2025-11-03-15-23-21 +0000,nypost,Chilling video shows UK mass train stabbing su...,https://nypost.com/2025/11/03/world-news/video...
138,43,2025-11-03-19-16-00 +0000,wsj,Federal Reserve governor Lisa Cook said she su...,https://www.wsj.com/economy/central-banking/em...
121,42,2025-11-03-20-07-00 +0000,wsj,The U.S. said it would use emergency funds to ...,https://www.wsj.com/politics/policy/u-s-to-pay...
185,35,2025-11-03-16-58-24 +0000,nypost,"Medieval tower partially collapses in Rome, le...",https://nypost.com/2025/11/03/world-news/medie...
213,35,2025-11-03-15-09-25 +0000,bbc,Israeli military's ex-top lawyer arrested over...,https://www.bbc.com/news/articles/cy0kpd97qqko...
221,33,2025-11-03-14-32-42 +0000,nypost,Antisemitic activist Linda Sarsour vows to hol...,https://nypost.com/2025/11/03/us-news/linda-sa...
234,33,2025-11-03-13-25-51 +0000,nypost,"Lone survivor of Air India crash speaks out, r...",https://nypost.com/2025/11/03/world-news/lone-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
